# Introduction 

MongoDB est une base de données open source codée en C++ basée sur un concept de stockage sous la forme de documents au format JSON. Le grand avantage de ce système est l'optimisation de la mémoire. Dans une base relationelle, chaque colonne doit être définie au préalable avec une empreinte mémoire et un type de donnée. Dans une base MongoDB si le champ n'est pas présent, il n'apparait pas dans un document et n'impacte pas la mémoire, alors qu'en SQL la place mémoire est utilisée même si le champ est absent, pour spécifier que la valeur est null.

## Les avantages 

- Optimisé pour le multi-machines et la réplication de données ;
- Pas besoin de jointures entre les tables compte tenu du modèle de données sous forme de documents ;
- Un index est créé sur chaque clé pour une rapidité de requêtes ;
- Un langage de requêtage aussi puissant que le SQL ;
- Optimisation de la mémoire .


## Concepts basiques



- Database : une database est un regroupement de collections. Chaque database possède son propre système de fichiers et sa propre authentification. Elle a le même rôle qu'une database en MySQL.
- Collection : une collection est un regroupement de documents. C'est une table en MySQL, la principale différence étant qu'elle ne définit pas un schéma de données fixe. Les documents présents dans une collection n'ont pas forcément tous les mêmes champs.
- Document : un document est un objet JSON stocké sous la forme de plusieurs paires clé:valeur. C'est l'équivalent d'une ligne dans une table SQL.
- Champ : Un champ est l'équivalent d'une colonne en SQL. Il permet de faire des requêtes.

## Identifiants

Tous les documents possèdent un identifiant unique, ce qui permet de le retrouver très efficacement. L'identifiant peut être spécifié lors de l'ajout d'un nouveau document (nom+prenom, adresse email, url, etc). Dans le cas ou aucun identifiant n'est précisé, MongoDB se charge d'en ajouter un. Il est composé d'un nombre stocké sur 12 bytes au format hexadécimal :

- Les 4 premiers bytes sont le timestamp de l'ajout du document
- Les 3 suivants correspondent à l'identifiant de la machine
- Les 2 suivants l'identifiant du processus
- Les 3 derniers sont une valeur incrémentale


## Les types de données

Une base de données MongoDB permet de stocker un grand volume de données hétérogènes sans imposer un modèle de données fixe pour tous les documents. Il est conseillé, comme vu plus haut, de bien définir la structure globale pour garder une cohérence tout au long des développements.

- Integer : entier relatif stocké sur 32 ou 64 bits.
- Double : nombre décimal stocké sur 64 bits.
- String : chaine de caractère (encodée en utf-8)
- Booléen : True ou False
- Object : sous-objet stocké au format JSON
- Date : date au format UNIX (nombre de ms écoulées depuis le 1er janvier 1970) stockée sur 64 bits.
- Array : stocker une liste d'éléments au format atomique ou d'objets

D'autres types sont disponibles et vous pouvez les trouver https://docs.mongodb.com/manual/reference/bson-types/

# Installation


L'installation peut se faire de plusieurs manières.

- Directement depuis les sources ou à partir de packages.  
Liens vers le tutorial https://docs.mongodb.com/manual/tutorial/install-mongodb-on-ubuntu/

Sur une Debian 9 Stretch:

```bash
export http_proxy=http://147.215.1.189:3128
export https_proxy=http://147.215.1.189:3128
apt-get update
apt-get install -y mongodb-org
```

Démarrage du service

Vérifier que le service Mongo est démarré

```bash
> service mongodb status


● mongodb.service - An object/document-oriented database
   Loaded: loaded (/lib/systemd/system/mongodb.service; enabled; vendor preset:
   Active: active (running) since Mon 2018-11-25 13:51:14 CET; 18min ago
     Docs: man:mongod(1)
 Main PID: 22845 (mongod)
    Tasks: 16 (limit: 4915)
   CGroup: /system.slice/mongodb.service
           └─22845 /usr/bin/mongod --unixSocketPrefix=/run/mongodb --config /etc
```

ou le démarrer avec 
 
```bash
> service mongodb start
```


- Ou en instanciant un conteneur Docker. L'avantage de Docker est qu'il n'installe aucune dépendance sur votre machine et laisse son environnement propre. Lien vers le tutorial : https://hub.docker.com/_/mongo/


```bash
> docker run --name my-mongo -d -p 27017:27017 mongo
```

Pour vérifier que le conteneur est bien lancé on peut regarder les logs avec


```bash
> docker logs my-mongo
```

# Connexion


Pour se connecter à une base Mongo deux solutions sont possibles. En ligne de commande ou via un gestionnaire de BDD comme Robo3T https://robomongo.org/ . Dans les deux cas, la syntaxe Mongo est utilisée pour effectuer des requêtes. L'avantage de Robo3T est qu'il possède une interface permettant de visualiser très simplement les données.

Dans un terminal utilisateur standard, la commande mongo permet d'obtenir un shell interactif:

```bash
student@debian:~$ mongo
MongoDB shell version: 3.2.11
connecting to: test
>
```

Le port par défaut de Mongo est le 27017.

# Création d'un modèle de données


La création d'un modèle de données clair et adapté est une tâche importante et primordiale. Ce modèle de données doit être réfléchi à court et long terme et doit prendre en compte la capacité de stockage et les besoins métiers.

## Database


A partir du shell Mongo, on peut afficher les databases disponibles. Au démarrage, aucune n'est créée:
```
> show dbs
admin   0.000GB
config  0.000GB
local   0.000GB
```

Aucun database n'est présente
```
> use test
switched to db test
```

Pour supprimer définitivement une database:

```
> db.dropDatabase()
```
Comme vous pouvez le deviner cette commande est à utiliser avec précaution.

## Collection

Les collections correspondent aux tables en SQL. Elles sont des sous-ensembles d'une database. Pour créer une collection il faut auparavant s'être référencé sur une database.

```
show dbs
use <YOUR_DB_NAME>
db.createCollection(<YOUR_COLLECTION_NAME>)
show collections
```

Comme pour les databases on peut vouloir supprimer définitivement une collection.

```
db.<YOUR_COLLECTION_NAME>.drop()
show collections
```


## Document

Insertion
Un document (objet JSON) est un sous-ensemble d'une collection qui est lui même une sous-partie d'une database. Pour insérer un document il faut donc se référencer sur une database et sur la collection souhaitée.

```
use <YOUR_DB_NAME>
db.createCollection(<YOUR_COLLECTION_NAME>)
show collections
db.<YOUR_COLLECTION_NAME>.insert({
    firstname : "Thomas",
    lastname : "Shelby",
    position : "director",
    company : "Peaky Blinders"})
```

Si vous ne précisez pas d'identifiant unique (id du document), MongoDB se charge de le remplir avec les règles définies précédement. Une bonne pratique est de trouver une règle permettant de retrouver facilement et efficacement un document sans avoir à faire une requête complexe et obliger la base à rechercher dans ses champs. Une technique est de prendre le hash d'une combinaison des champs qui permet de créer une clé unique SHA128(firstname+lastname+position) par exemple.

```
use <YOUR_DB_NAME>
show collections
db.<YOUR_COLLECTION_NAME>.insert({
    firstname : "Thomas",
    lastname : "Shelby",
    position : "CEO",
    gender : "Male",
    age : 35,
    description : "Thomas 'Tommy' Michael Shelby M.P. OBE, is the leader of the Birmingham criminal gang Peaky Blinders and the patriarch of the Shelby Family. His experiences during and after the First World War have left him disillusioned and determined to move his family up in the world.",
    nicknames : ["Tom", "Tommy", "Thomas"],
    company : "Peaky Blinders",
    episodes : [1,2,4,5,6]
    })
```
Pour des soucis de performances, si un grand nombre de documents doivent être insérés très rapidement sans surcharger les appels réseaux, il est possible de passer une liste JSON d'objets à la fonction insert.

```
db.<YOUR_COLLECTION_NAME>.insert([
{
    firstname : "Arthur",
    lastname : "Shelby",
    position : "Associate",
    gender : "Male",
    age : 38,
    description : "Arthur Shelby Jr. is the eldest of the Shelby siblings and the tough member of Peaky Blinders, the Deputy Vice President Shelby Company Limited. He's also a member of the ICA.",
    company : "Peaky Blinders",
    episodes : [1,4,6]

},{
    firstname : "John",
    lastname : "Shelby",
    position : "Associate",
    gender : "Male",
    age : 30,
    description : "John Michael Shelby, also called Johnny or John Boy, was the third of Shelby siblings and a member of the Peaky Blinders.",
    nicknames : ["Johnny", "John Boy"],
    company : "Peaky Blinders",
    episodes : [4,5,6]
},{
    firstname : "Ada",
    lastname : "Thorne",
    position : "HR",
    gender : "Female",
    age : 28,
    description : "Ada Thorne is the fourth and only female of the Shelby sibling. She's the Head of Acquisitions of the Shelby Company Limited.",
    nicknames : ["Ada Shelby"],
    company : "Peaky Blinders",
    episodes : [1,2,6]
},{
    firstname : "Michael",
    lastname : "Gray",
    position : "Accounting",
    gender : "Male",
    age : 21,
    description : "Michael Gray is the son of Polly Shelby, his father is dead, and cousin of the Shelby siblings. He is the Chief Accountant in the Shelby Company Limited.",
    nicknames : ["Henry Johnson", "Jobbie Muncher", "Mickey"],
    company : "Peaky Blinders",
    episodes : [5,6]
},{
    firstname : "Polly",
    lastname : "Gray",
    gender : "Female",
    age : 45,
    position : "CFO",
    description : "Elizabeth Polly Gray (née Shelby) is the matriarch of the Shelby Family, aunt of the Shelby siblings, the treasurer of the Birmingham criminal gang, the Peaky Blinders, a certified accountant and company treasurer of Shelby Company Limited. ",
    nicknames : ["Aunt Polly", "Polly Gray", "Elizabeth Gray", "Polly Shelby", "Pol"],
    company : "Peaky Blinders",
    episodes : [1,2,5,6]
}])
```